<a href="https://colab.research.google.com/github/Anjasfedo/Code-as-a-Cryptography/blob/main/ecc_lsb_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Elliptic Curve Criptography

In [171]:
!pip install eciespy

In [172]:
CONTENT = "3348610401970005#christofer*derian*budianto#tegal#1997-03-04#laki-laki#b#jl.*pala*22*no.*30#005#017#mejasem*tengah#kramat#katholik#belum*kawin#pelajar/mahasiswa#wni#seumur*hidup"

In [173]:
from ecies.utils import generate_eth_key
from ecies import encrypt, decrypt

private_key = generate_eth_key()
public_key = private_key.public_key

public_key_hex = public_key.to_hex()
private_key_hex = private_key.to_hex()

print("Public Key:", public_key_hex)
print("Private Key:", private_key_hex)

ciphertext = encrypt(public_key_hex, CONTENT.encode())

print("Ciphertext:", ciphertext.hex())

decrypted_message = decrypt(private_key_hex, ciphertext)
print("Decrypted Message:", decrypted_message.decode())

Public Key: 0xe0f3b54568e7b1315ee1edd7b0104175e23ad99d0493ca1c95dd580a519de68b335125de1a6de00cc5cf408c65994f5b2499edf8199951d1f802c9541496d573
Private Key: 0xb3e15c21c6ed4d82d7da565a6400659e8b168c884ea7de8602bd077011810225
Ciphertext: 04dd2a4b6cc7fc8addc7162361b4857a3010c24233c99f5f933e59dddc90f1586c9e1f14522af9bedae880b11948bf54f6fba6be811e540b525e1d5cc716d14e73573386b2e3cce5ec2462369e9a656a48c2e31b7668352bfbc89a684de65a5c2a6e10ad4702dca6e7235e635bfe59d90561f79384b77140e608ce5647593f32d148b48742929335e88e8d720adde35a418cc1a743a2bdbc67b35e5401dcc04bfc4c0c8a9ddc579a9613f206f4aa802fa1fca58b863d3f2666d935f94b68e9d803b25570e42bb7fd48b82938b864bf2f00795be553cff04ea653fc3115546ce95c39bf04e6b100df09cab21c5ea19921bc8006869458fe837c2df0cf77f500e9cf7c55ffd0386505a4aecae3726c3c62b3ec
Decrypted Message: 3348610401970005#christofer*derian*budianto#tegal#1997-03-04#laki-laki#b#jl.*pala*22*no.*30#005#017#mejasem*tengah#kramat#katholik#belum*kawin#pelajar/mahasiswa#wni#seumur*hidup


# Least Significant Bit

In [174]:
!git clone https://github.com/RobinDavid/LSB-Steganography && cd LSB-Steganography && ls && pip install -r requirements.txt

fatal: destination path 'LSB-Steganography' already exists and is not an empty directory.


In [175]:
#!/usr/bin/env python
# coding:UTF-8
"""LSBSteg.py

Usage:
  LSBSteg.py encode -i <input> -o <output> -f <file>
  LSBSteg.py decode -i <input> -o <output>

Options:
  -h, --help                Show this help
  --version                 Show the version
  -f,--file=<file>          File to hide
  -i,--in=<input>           Input image (carrier)
  -o,--out=<output>         Output image (or extracted file)
"""

import cv2
import docopt
import numpy as np


class SteganographyException(Exception):
    pass


class LSBSteg():
    def __init__(self, im):
        self.image = im
        self.height, self.width, self.nbchannels = im.shape
        self.size = self.width * self.height

        self.maskONEValues = [1,2,4,8,16,32,64,128]
        #Mask used to put one ex:1->00000001, 2->00000010 .. associated with OR bitwise
        self.maskONE = self.maskONEValues.pop(0) #Will be used to do bitwise operations

        self.maskZEROValues = [254,253,251,247,239,223,191,127]
        #Mak used to put zero ex:254->11111110, 253->11111101 .. associated with AND bitwise
        self.maskZERO = self.maskZEROValues.pop(0)

        self.curwidth = 0  # Current width position
        self.curheight = 0 # Current height position
        self.curchan = 0   # Current channel position

    def put_binary_value(self, bits): #Put the bits in the image
        for c in bits:
            val = list(self.image[self.curheight,self.curwidth]) #Get the pixel value as a list
            if int(c) == 1:
                val[self.curchan] = int(val[self.curchan]) | self.maskONE #OR with maskONE
            else:
                val[self.curchan] = int(val[self.curchan]) & self.maskZERO #AND with maskZERO

            self.image[self.curheight,self.curwidth] = tuple(val)
            self.next_slot() #Move "cursor" to the next space

    def next_slot(self):#Move to the next slot were information can be taken or put
        if self.curchan == self.nbchannels-1: #Next Space is the following channel
            self.curchan = 0
            if self.curwidth == self.width-1: #Or the first channel of the next pixel of the same line
                self.curwidth = 0
                if self.curheight == self.height-1:#Or the first channel of the first pixel of the next line
                    self.curheight = 0
                    if self.maskONE == 128: #Mask 1000000, so the last mask
                        raise SteganographyException("No available slot remaining (image filled)")
                    else: #Or instead of using the first bit start using the second and so on..
                        self.maskONE = self.maskONEValues.pop(0)
                        self.maskZERO = self.maskZEROValues.pop(0)
                else:
                    self.curheight +=1
            else:
                self.curwidth +=1
        else:
            self.curchan +=1

    def read_bit(self): #Read a single bit int the image
        val = self.image[self.curheight,self.curwidth][self.curchan]
        val = int(val) & self.maskONE
        self.next_slot()
        if val > 0:
            return "1"
        else:
            return "0"

    def read_byte(self):
        return self.read_bits(8)

    def read_bits(self, nb): #Read the given number of bits
        bits = ""
        for i in range(nb):
            bits += self.read_bit()
        return bits

    def byteValue(self, val):
        return self.binary_value(val, 8)

    def binary_value(self, val, bitsize): #Return the binary value of an int as a byte
        binval = bin(val)[2:]
        if len(binval) > bitsize:
            raise SteganographyException("binary value larger than the expected size")
        while len(binval) < bitsize:
            binval = "0"+binval
        return binval

    def encode_text(self, txt):
        l = len(txt)
        binl = self.binary_value(l, 16) #Length coded on 2 bytes so the text size can be up to 65536 bytes long
        self.put_binary_value(binl) #Put text length coded on 4 bytes
        for char in txt: #And put all the chars
            c = ord(char)
            self.put_binary_value(self.byteValue(c))
        return self.image

    def decode_text(self):
        ls = self.read_bits(16) #Read the text size in bytes
        l = int(ls,2)
        i = 0
        unhideTxt = ""
        while i < l: #Read all bytes of the text
            tmp = self.read_byte() #So one byte
            i += 1
            unhideTxt += chr(int(tmp,2)) #Every chars concatenated to str
        return unhideTxt

    def encode_image(self, imtohide):
        w = imtohide.width
        h = imtohide.height
        if self.width*self.height*self.nbchannels < w*h*imtohide.channels:
            raise SteganographyException("Carrier image not big enough to hold all the datas to steganography")
        binw = self.binary_value(w, 16) #Width coded on to byte so width up to 65536
        binh = self.binary_value(h, 16)
        self.put_binary_value(binw) #Put width
        self.put_binary_value(binh) #Put height
        for h in range(imtohide.height): #Iterate the hole image to put every pixel values
            for w in range(imtohide.width):
                for chan in range(imtohide.channels):
                    val = imtohide[h,w][chan]
                    self.put_binary_value(self.byteValue(int(val)))
        return self.image


    def decode_image(self):
        width = int(self.read_bits(16),2) #Read 16bits and convert it in int
        height = int(self.read_bits(16),2)
        unhideimg = np.zeros((width,height, 3), np.uint8) #Create an image in which we will put all the pixels read
        for h in range(height):
            for w in range(width):
                for chan in range(unhideimg.channels):
                    val = list(unhideimg[h,w])
                    val[chan] = int(self.read_byte(),2) #Read the value
                    unhideimg[h,w] = tuple(val)
        return unhideimg

    def encode_binary(self, data):
        l = len(data)
        if self.width*self.height*self.nbchannels < l+64:
            raise SteganographyException("Carrier image not big enough to hold all the datas to steganography")
        self.put_binary_value(self.binary_value(l, 64))
        for byte in data:
            byte = byte if isinstance(byte, int) else ord(byte) # Compat py2/py3
            self.put_binary_value(self.byteValue(byte))
        return self.image

    def decode_binary(self):
        l = int(self.read_bits(64), 2)
        output = b""
        for i in range(l):
            output += bytearray([int(self.read_byte(),2)])
        return output


def main():
    args = docopt.docopt(__doc__, version="0.2")
    in_f = args["--in"]
    out_f = args["--out"]
    in_img = cv2.imread(in_f)
    steg = LSBSteg(in_img)
    lossy_formats = ["jpeg", "jpg"]

    if args['encode']:
        #Handling lossy format
        out_f, out_ext = out_f.split(".")
        if out_ext in lossy_formats:
            out_f = out_f + ".png"
            print("Output file changed to ", out_f)

        data = open(args["--file"], "rb").read()
        res = steg.encode_binary(data)
        cv2.imwrite(out_f, res)

    elif args["decode"]:
        raw = steg.decode_binary()
        with open(out_f, "wb") as f:
            f.write(raw)

# if __name__=="__main__":
#     main()

# Lampel Ziv Welch

In [176]:
!git clone https://github.com/joeatwork/python-lzw

fatal: destination path 'python-lzw' already exists and is not an empty directory.


In [177]:
!cd /content/python-lzw && mv lzw/ /content/ && python setup.py install

mv: cannot stat 'lzw/': No such file or directory


In [178]:
import lzw

# Example text to compress (can be any string)
original_text = CONTENT

# Convert the original text into bytes (UTF-8 encoding)
original_bytes = original_text.encode('utf-8')

# Part 1: Compression (in-memory)
compressed = lzw.compress(original_bytes)

# Convert compressed generator to a list to see its contents
compressed_list = list(compressed)

# Part 2: Decompression (in-memory)
decompressed_chunks = lzw.decompress(compressed_list)

# Join the decompressed chunks into a single byte string
decompressed_bytes = b"".join(decompressed_chunks)

# Convert decompressed bytes back to a string
decompressed_text = decompressed_bytes.decode('utf-8')

# Print each variable
print("Original Text (String):", original_text)
print("Original Bytes:", original_bytes)
print("Compressed List (Byte Chunks):", compressed_list)
print("Decompressed Bytes:", decompressed_bytes)
print("Decompressed Text (String):", decompressed_text)

# Part 3: Print sizes and results for comparison
original_size = len(original_bytes)
compressed_size = len(b"".join(compressed_list))  # Join to get compressed byte size
decompressed_size = len(decompressed_bytes)

# Print the sizes in bytes
print(f"\nOriginal size: {original_size} bytes")
print(f"Compressed size: {compressed_size} bytes")
print(f"Decompressed size: {decompressed_size} bytes")

# Check if decompression was successful
if decompressed_text == original_text:
    print("Decompression was successful!")
else:
    print("Decompression failed.")

Original Text (String): 3348610401970005#christofer*derian*budianto#tegal#1997-03-04#laki-laki#b#jl.*pala*22*no.*30#005#017#mejasem*tengah#kramat#katholik#belum*kawin#pelajar/mahasiswa#wni#seumur*hidup
Original Bytes: b'3348610401970005#christofer*derian*budianto#tegal#1997-03-04#laki-laki#b#jl.*pala*22*no.*30#005#017#mejasem*tengah#kramat#katholik#belum*kawin#pelajar/mahasiswa#wni#seumur*hidup'
Compressed List (Byte Chunks): [b'\x19', b'\x8c', b'\xc6', b'\x83', b'\x81', b'\xb0', b'\xc4', b'`', b'4', b'\x18', b'\x0c', b'G', b'#', b'q', b'\x84', b'8', b'j', b'#', b'1', b'\x9a', b'\x0e', b'F', b'\x93', b'\x99', b'\xd0', b'\xde', b'f', b'2', b'\x9c', b'\x85', b'F', b'H', b'\xd1', b'\xa4', b'\xc2', b'n', b'\x15', b'\x18', b'\x8e', b'\xa6', b'H', b'\xf9', b'\xba', b'.', b'#', b':', b'\x19', b'L', b'\xe6', b'\x13', b'`', b'\x8e', b'\x17', b'\x0c', b'\x16', b'\x8c', b'\x06', b's', b'!', b'\xa0', b'\x8c', b'\xd8', b'a', b'5', b'\x9a', b'E', b'\xb3', b'\x89', b'\xd0', b'\x8c', b'\xc4', b'#', b'

# Huffman Encoding

In [179]:
import heapq
from collections import Counter

# Class for Huffman Tree Nodes
class HuffmanNode:
    def __init__(self, char=None, freq=0, left=None, right=None):
        self.char = char
        self.freq = freq
        self.left = left
        self.right = right

    def __lt__(self, other):
        return self.freq < other.freq

# Function to build the Huffman Tree
def build_huffman_tree(frequency):
    heap = [HuffmanNode(char, freq) for char, freq in frequency.items()]
    heapq.heapify(heap)

    while len(heap) > 1:
        node1 = heapq.heappop(heap)
        node2 = heapq.heappop(heap)
        merged = HuffmanNode(freq=node1.freq + node2.freq, left=node1, right=node2)
        heapq.heappush(heap, merged)

    return heap[0]  # Return the root of the tree

# Function to generate Huffman codes from the Huffman Tree
def generate_huffman_codes(node, current_code="", huffman_codes={}):
    if node is None:
        return

    if node.char is not None:
        huffman_codes[node.char] = current_code

    generate_huffman_codes(node.left, current_code + "0", huffman_codes)
    generate_huffman_codes(node.right, current_code + "1", huffman_codes)

    return huffman_codes

# Function to compress the data using Huffman Encoding
def huffman_compress(data):
    # Count the frequency of each byte
    frequency = Counter(data)

    # Build Huffman Tree
    huffman_tree = build_huffman_tree(frequency)

    # Generate Huffman Codes
    huffman_codes = generate_huffman_codes(huffman_tree)

    # Encode data into its binary representation
    encoded_data = "".join(huffman_codes[byte] for byte in data)

    # Pad the encoded data to make its length a multiple of 8
    padding = 8 - len(encoded_data) % 8
    encoded_data += "0" * padding
    padding_info = "{0:08b}".format(padding)

    # Convert binary data to bytes
    compressed_data = bytearray()
    compressed_data.append(int(padding_info, 2))  # First byte stores the padding information
    for i in range(0, len(encoded_data), 8):
        byte = encoded_data[i:i+8]
        compressed_data.append(int(byte, 2))

    return compressed_data, huffman_tree

# Function to decompress the data using Huffman Encoding
def huffman_decompress(compressed_data, huffman_tree):
    # Convert the compressed data to binary string
    encoded_data = ""
    for byte in compressed_data[1:]:
        encoded_data += "{0:08b}".format(byte)

    # Get the padding information from the first byte
    padding = compressed_data[0]
    encoded_data = encoded_data[:-padding]  # Remove padding

    # Traverse the Huffman tree to decode the binary string
    decoded_data = bytearray()
    node = huffman_tree
    for bit in encoded_data:
        if bit == "0":
            node = node.left
        else:
            node = node.right

        if node.left is None and node.right is None:  # Leaf node
            decoded_data.append(node.char)
            node = huffman_tree

    return decoded_data


# Example Usage of Huffman Compression/Decompression

# Step 1: The original plaintext data (as a byte string)
plaintext = CONTENT.encode()

# Print the original data
print("Original Data (as bytes):", plaintext)
print("Original Data (as string):", plaintext.decode())

# Step 2: Compress the plaintext data using Huffman encoding
compressed_data, huffman_tree = huffman_compress(plaintext)
print("Compressed Data (in bytes):", compressed_data)

# Step 3: Decompress the compressed data back to the original plaintext
decompressed_data = huffman_decompress(compressed_data, huffman_tree)
print("Decompressed Data (as bytes):", decompressed_data)
print("Decompressed Data (as string):", decompressed_data.decode())

# Step 4: Verify that the decompressed data matches the original data
if decompressed_data == plaintext:
    print("Decompression was successful!")
else:
    print("Decompression failed.")

Original Data (as bytes): b'3348610401970005#christofer*derian*budianto#tegal#1997-03-04#laki-laki#b#jl.*pala*22*no.*30#005#017#mejasem*tengah#kramat#katholik#belum*kawin#pelajar/mahasiswa#wni#seumur*hidup'
Original Data (as string): 3348610401970005#christofer*derian*budianto#tegal#1997-03-04#laki-laki#b#jl.*pala*22*no.*30#005#017#mejasem*tengah#kramat#katholik#belum*kawin#pelajar/mahasiswa#wni#seumur*hidup
Compressed Data (in bytes): bytearray(b'\x02\xdfy>\xa8\x7f\x12G\xf7S\x11\x1e\xfb\xedYu\xcf\xc6)\x91\xd52\xe6\xc3\xa3usi\xf9\xd3OD;\xfb\xae\xa6\xf8\xde\xf8\x93\x80\xe1\xef\x03\x87\xed9\x90\xa2;\x10#\xf3\xe0\xdb\xca#\xbctG\xbe\x8f\xe9\xea\xa3%iQ\x9a[\xd1Wc&\xa6}\x835x\x0f\x8e\xd1\x01\xb5\x1c\x19^\xdd\xb2\x01d\xe5\x85J\x8a\xddr\x9e\x96\xcf\xcbF\xd5\xb9\x15>\xad\xd8')
Decompressed Data (as bytes): bytearray(b'3348610401970005#christofer*derian*budianto#tegal#1997-03-04#laki-laki#b#jl.*pala*22*no.*30#005#017#mejasem*tengah#kramat#katholik#belum*kawin#pelajar/mahasiswa#wni#seumur*hidup')

# Test Image

In [180]:
import os
# Directory where the image will be saved
ori_dir_img = "ori-image"

# Create the ori_dir_img if it doesn't exist
if not os.path.exists(ori_dir_img):
    os.makedirs(ori_dir_img)

stego_dir_img = "stego-image"

# Create the stego_dir_img if it doesn't exist
if not os.path.exists(stego_dir_img):
    os.makedirs(stego_dir_img)

## Lena Image

In [181]:
import requests
from PIL import Image
import io

# URL to the raw image file
url = "https://raw.githubusercontent.com/mikolalysenko/lena/master/lena.png"

# Download the image
response = requests.get(url)
if response.status_code == 200:
    # Load the image using PIL
    lena_image = Image.open(io.BytesIO(response.content))
    lena_image.show()  # Display the image (optional)
    lena_image.save("ori-image/lena.png")  # Save the image locally
else:
    print("Failed to download the image.")

In [182]:
LENA_IMG = 'ori-image/lena.png'

if not os.path.exists(LENA_IMG):
    raise FileNotFoundError(f"Image not found at {LENA_IMG}")

# Steganography Metric

## MSE

In [183]:
def calculate_mse(original, stego):
    return np.mean((original - stego) ** 2)

## PSNR

In [184]:
def calculate_psnr(target, ori_image=LENA_IMG):
    # Read images
    original = cv2.imread(ori_image)
    stego = cv2.imread(f'{stego_dir_img}/{target}')

    if original is None:
        raise ValueError(f"Failed to load original image from {ori_image}. Ensure the file exists and is a valid image format.")
    if stego is None:
        raise ValueError(f"Failed to load stego image from {stego_dir_img}/{target}. Ensure the file exists and is a valid image format.")

    # Calculate MSE using the separated function
    mse = calculate_mse(original, stego)
    if mse == 0:  # If images are identical
        return float('inf')

    # Calculate PSNR using the 10 log10 version
    max_pixel_value = 255.0
    psnr = 10 * np.log10((max_pixel_value ** 2) / mse)
    # print(f'PSNR: {psnr}')
    return psnr

## SSIM

In [185]:
from skimage.metrics import structural_similarity as ssim

def calculate_ssim(original, stego):
    # Convert images to grayscale
    original_gray = cv2.cvtColor(original, cv2.COLOR_BGR2GRAY)
    stego_gray = cv2.cvtColor(stego, cv2.COLOR_BGR2GRAY)

    # Calculate SSIM
    ssim_value, _ = ssim(original_gray, stego_gray, full=True)
    return ssim_value

## Get it together

In [186]:
def calculate_stego_metrics(target, ori_image=LENA_IMG):
    # Read images
    original = cv2.imread(ori_image)
    stego = cv2.imread(f'{stego_dir_img}/{target}')

    if original is None:
        raise ValueError(f"Failed to load original image from {ori_image}. Ensure the file exists and is a valid image format.")
    if stego is None:
        raise ValueError(f"Failed to load stego image from {stego_dir_img}/{target}. Ensure the file exists and is a valid image format.")

    # Calculate metrics
    mse_value = calculate_mse(original, stego)
    psnr_value = calculate_psnr(target)
    ssim_value = calculate_ssim(original, stego)

    # Print results
    print(f'Metrics between original ({ori_image}) and stego image ({target}):')
    print(f'MSE: {mse_value}')
    print(f'PSNR: {psnr_value}')
    print(f'SSIM: {ssim_value}')

    return mse_value, psnr_value, ssim_value

# LSB Helper

In [187]:
def embed_lsb(target, content, ori_img = LENA_IMG):
  steg = LSBSteg(cv2.imread(ori_img))

  img_encoded = steg.encode_text(content)

  lsb_img = f"{stego_dir_img}/{target}"

  cv2.imwrite(lsb_img, img_encoded)

In [188]:
def extract_lsb(target):
  im = cv2.imread(f"{stego_dir_img}/{target}")

  steg = LSBSteg(im)

  content = steg.decode_text()

  return content

# LSB Only

In [189]:
embed_lsb("lsb_only.png", CONTENT)

In [190]:
extract_lsb("lsb_only.png")

'3348610401970005#christofer*derian*budianto#tegal#1997-03-04#laki-laki#b#jl.*pala*22*no.*30#005#017#mejasem*tengah#kramat#katholik#belum*kawin#pelajar/mahasiswa#wni#seumur*hidup'

In [191]:
extract_lsb("lsb_only.png") == CONTENT

True

## Stego Metric

In [192]:
calculate_stego_metrics("lsb_only.png")

Metrics between original (ori-image/lena.png) and stego image (lsb_only.png):
MSE: 0.0009028116861979166
PSNR: 78.57483188820159
SSIM: 0.9999987952461553


(0.0009028116861979166, 78.57483188820159, 0.9999987952461553)

# LSB with ECC

In [193]:
ciphertext

b'\x04\xdd*Kl\xc7\xfc\x8a\xdd\xc7\x16#a\xb4\x85z0\x10\xc2B3\xc9\x9f_\x93>Y\xdd\xdc\x90\xf1Xl\x9e\x1f\x14R*\xf9\xbe\xda\xe8\x80\xb1\x19H\xbfT\xf6\xfb\xa6\xbe\x81\x1eT\x0bR^\x1d\\\xc7\x16\xd1NsW3\x86\xb2\xe3\xcc\xe5\xec$b6\x9e\x9aejH\xc2\xe3\x1bvh5+\xfb\xc8\x9ahM\xe6Z\\*n\x10\xadG\x02\xdc\xa6\xe7#^c[\xfeY\xd9\x05a\xf7\x93\x84\xb7q@\xe6\x08\xceVGY?2\xd1H\xb4\x87B\x92\x935\xe8\x8e\x8dr\n\xdd\xe3ZA\x8c\xc1\xa7C\xa2\xbd\xbcg\xb3^T\x01\xdc\xc0K\xfcL\x0c\x8a\x9d\xdcW\x9a\x96\x13\xf2\x06\xf4\xaa\x80/\xa1\xfc\xa5\x8b\x86=?&f\xd95\xf9Kh\xe9\xd8\x03\xb2Up\xe4+\xb7\xfdH\xb8)8\xb8d\xbf/\x00y[\xe5S\xcf\xf0N\xa6S\xfc1\x15Tl\xe9\\9\xbf\x04\xe6\xb1\x00\xdf\t\xca\xb2\x1c^\xa1\x99!\xbc\x80\x06\x86\x94X\xfe\x83|-\xf0\xcfw\xf5\x00\xe9\xcf|U\xff\xd08e\x05\xa4\xae\xca\xe3rl<b\xb3\xec'

In [194]:
embed_lsb("lsb_w_ECC.png", ciphertext.hex())

In [195]:
extracted_data = extract_lsb("lsb_w_ECC.png")
extracted_data

'04dd2a4b6cc7fc8addc7162361b4857a3010c24233c99f5f933e59dddc90f1586c9e1f14522af9bedae880b11948bf54f6fba6be811e540b525e1d5cc716d14e73573386b2e3cce5ec2462369e9a656a48c2e31b7668352bfbc89a684de65a5c2a6e10ad4702dca6e7235e635bfe59d90561f79384b77140e608ce5647593f32d148b48742929335e88e8d720adde35a418cc1a743a2bdbc67b35e5401dcc04bfc4c0c8a9ddc579a9613f206f4aa802fa1fca58b863d3f2666d935f94b68e9d803b25570e42bb7fd48b82938b864bf2f00795be553cff04ea653fc3115546ce95c39bf04e6b100df09cab21c5ea19921bc8006869458fe837c2df0cf77f500e9cf7c55ffd0386505a4aecae3726c3c62b3ec'

In [196]:
extracted_data == ciphertext.hex()

True

In [197]:
extracted_data = bytes.fromhex(extracted_data)
extracted_data

b'\x04\xdd*Kl\xc7\xfc\x8a\xdd\xc7\x16#a\xb4\x85z0\x10\xc2B3\xc9\x9f_\x93>Y\xdd\xdc\x90\xf1Xl\x9e\x1f\x14R*\xf9\xbe\xda\xe8\x80\xb1\x19H\xbfT\xf6\xfb\xa6\xbe\x81\x1eT\x0bR^\x1d\\\xc7\x16\xd1NsW3\x86\xb2\xe3\xcc\xe5\xec$b6\x9e\x9aejH\xc2\xe3\x1bvh5+\xfb\xc8\x9ahM\xe6Z\\*n\x10\xadG\x02\xdc\xa6\xe7#^c[\xfeY\xd9\x05a\xf7\x93\x84\xb7q@\xe6\x08\xceVGY?2\xd1H\xb4\x87B\x92\x935\xe8\x8e\x8dr\n\xdd\xe3ZA\x8c\xc1\xa7C\xa2\xbd\xbcg\xb3^T\x01\xdc\xc0K\xfcL\x0c\x8a\x9d\xdcW\x9a\x96\x13\xf2\x06\xf4\xaa\x80/\xa1\xfc\xa5\x8b\x86=?&f\xd95\xf9Kh\xe9\xd8\x03\xb2Up\xe4+\xb7\xfdH\xb8)8\xb8d\xbf/\x00y[\xe5S\xcf\xf0N\xa6S\xfc1\x15Tl\xe9\\9\xbf\x04\xe6\xb1\x00\xdf\t\xca\xb2\x1c^\xa1\x99!\xbc\x80\x06\x86\x94X\xfe\x83|-\xf0\xcfw\xf5\x00\xe9\xcf|U\xff\xd08e\x05\xa4\xae\xca\xe3rl<b\xb3\xec'

In [198]:
decrypted_message = decrypt(private_key_hex, extracted_data)
print("Decrypted Message:", decrypted_message.decode())

Decrypted Message: 3348610401970005#christofer*derian*budianto#tegal#1997-03-04#laki-laki#b#jl.*pala*22*no.*30#005#017#mejasem*tengah#kramat#katholik#belum*kawin#pelajar/mahasiswa#wni#seumur*hidup


In [199]:
decrypted_message.decode() == CONTENT

True

## Stego Metric

In [200]:
calculate_stego_metrics("lsb_w_ECC.png")

Metrics between original (ori-image/lena.png) and stego image (lsb_w_ECC.png):
MSE: 0.002750396728515625
PSNR: 73.73685018100143
SSIM: 0.9999927734258244


(0.002750396728515625, 73.73685018100143, 0.9999927734258244)

# LSB with LZW

In [201]:
compressed_list

[b'\x19',
 b'\x8c',
 b'\xc6',
 b'\x83',
 b'\x81',
 b'\xb0',
 b'\xc4',
 b'`',
 b'4',
 b'\x18',
 b'\x0c',
 b'G',
 b'#',
 b'q',
 b'\x84',
 b'8',
 b'j',
 b'#',
 b'1',
 b'\x9a',
 b'\x0e',
 b'F',
 b'\x93',
 b'\x99',
 b'\xd0',
 b'\xde',
 b'f',
 b'2',
 b'\x9c',
 b'\x85',
 b'F',
 b'H',
 b'\xd1',
 b'\xa4',
 b'\xc2',
 b'n',
 b'\x15',
 b'\x18',
 b'\x8e',
 b'\xa6',
 b'H',
 b'\xf9',
 b'\xba',
 b'.',
 b'#',
 b':',
 b'\x19',
 b'L',
 b'\xe6',
 b'\x13',
 b'`',
 b'\x8e',
 b'\x17',
 b'\x0c',
 b'\x16',
 b'\x8c',
 b'\x06',
 b's',
 b'!',
 b'\xa0',
 b'\x8c',
 b'\xd8',
 b'a',
 b'5',
 b'\x9a',
 b'E',
 b'\xb3',
 b'\x89',
 b'\xd0',
 b'\x8c',
 b'\xc4',
 b'#',
 b'5',
 b'\x1b',
 b'\x05',
 b'\xc2',
 b'\xa3',
 b'\x84',
 b'\xb4',
 b'\xc2',
 b'*',
 b'\x19',
 b'\x0c',
 b'\x85',
 b'F',
 b'\xe3',
 b'}',
 b'\x10',
 b'f',
 b'0',
 b'\x11',
 b'\xc3',
 b'\xea',
 b'C',
 b'\x11',
 b'\xb8',
 b'\x8c',
 b'\xda',
 b'e',
 b'5',
 b'\x18',
 b'N',
 b'f',
 b'S',
 b'h',
 b'\xaa',
 b'T',
 b'n',
 b'\x96',
 b'\x1a',
 b'\x04',
 b'f',
 b'\xb3',

In [202]:
embed_lsb("lsb_w_LZW.png", compressed_list)

In [203]:
extracted_data = extract_lsb("lsb_w_LZW.png")
extracted_data

'\x19\x8cÆ\x83\x81°Ä`4\x18\x0cG#q\x848j#1\x9a\x0eF\x93\x99ÐÞf2\x9c\x85FHÑ¤Ân\x15\x18\x8e¦Hùº.#:\x19Læ\x13`\x8e\x17\x0c\x16\x8c\x06s!\xa0\x8cØa5\x9aE³\x89Ð\x8cÄ#5\x1b\x05Â£\x84´Â*\x19\x0c\x85Fã}\x10f0\x11ÃêC\x11¸\x8cÚe5\x18NfShªTn\x96\x1a\x04f³\x91\x84Úa:Y-&\x83y°Òk\x9f\x99M\x87Zñ¬Âw4\x9b\x84g\x0b\x99\x86´r\x17Ú\r\x15¸©ÜÂ#;\x9b\x8d":åÔë\x1b4\x1aL\x87S\x84\x00'

In [204]:
byte_list = [bytes([ord(char)]) for char in extracted_data]
byte_list

[b'\x19',
 b'\x8c',
 b'\xc6',
 b'\x83',
 b'\x81',
 b'\xb0',
 b'\xc4',
 b'`',
 b'4',
 b'\x18',
 b'\x0c',
 b'G',
 b'#',
 b'q',
 b'\x84',
 b'8',
 b'j',
 b'#',
 b'1',
 b'\x9a',
 b'\x0e',
 b'F',
 b'\x93',
 b'\x99',
 b'\xd0',
 b'\xde',
 b'f',
 b'2',
 b'\x9c',
 b'\x85',
 b'F',
 b'H',
 b'\xd1',
 b'\xa4',
 b'\xc2',
 b'n',
 b'\x15',
 b'\x18',
 b'\x8e',
 b'\xa6',
 b'H',
 b'\xf9',
 b'\xba',
 b'.',
 b'#',
 b':',
 b'\x19',
 b'L',
 b'\xe6',
 b'\x13',
 b'`',
 b'\x8e',
 b'\x17',
 b'\x0c',
 b'\x16',
 b'\x8c',
 b'\x06',
 b's',
 b'!',
 b'\xa0',
 b'\x8c',
 b'\xd8',
 b'a',
 b'5',
 b'\x9a',
 b'E',
 b'\xb3',
 b'\x89',
 b'\xd0',
 b'\x8c',
 b'\xc4',
 b'#',
 b'5',
 b'\x1b',
 b'\x05',
 b'\xc2',
 b'\xa3',
 b'\x84',
 b'\xb4',
 b'\xc2',
 b'*',
 b'\x19',
 b'\x0c',
 b'\x85',
 b'F',
 b'\xe3',
 b'}',
 b'\x10',
 b'f',
 b'0',
 b'\x11',
 b'\xc3',
 b'\xea',
 b'C',
 b'\x11',
 b'\xb8',
 b'\x8c',
 b'\xda',
 b'e',
 b'5',
 b'\x18',
 b'N',
 b'f',
 b'S',
 b'h',
 b'\xaa',
 b'T',
 b'n',
 b'\x96',
 b'\x1a',
 b'\x04',
 b'f',
 b'\xb3',

In [205]:
compressed_list == byte_list

True

In [206]:
# Part 2: Decompression (in-memory)
decompressed_chunks = lzw.decompress(byte_list)

# Join the decompressed chunks into a single byte string
decompressed_bytes = b"".join(decompressed_chunks)

# Convert decompressed bytes back to a string
decompressed_text = decompressed_bytes.decode('utf-8')

print("Decompressed Text (String):", decompressed_text)

Decompressed Text (String): 3348610401970005#christofer*derian*budianto#tegal#1997-03-04#laki-laki#b#jl.*pala*22*no.*30#005#017#mejasem*tengah#kramat#katholik#belum*kawin#pelajar/mahasiswa#wni#seumur*hidup


In [207]:
decompressed_text == CONTENT

True

## Stego Metric

In [208]:
calculate_stego_metrics("lsb_w_LZW.png")

Metrics between original (ori-image/lena.png) and stego image (lsb_w_LZW.png):
MSE: 0.0008722941080729166
PSNR: 78.72417421832482
SSIM: 0.9999988544716335


(0.0008722941080729166, 78.72417421832482, 0.9999988544716335)

# LSB with ECC, LZW

## ECC

In [209]:
ciphertext.hex()

'04dd2a4b6cc7fc8addc7162361b4857a3010c24233c99f5f933e59dddc90f1586c9e1f14522af9bedae880b11948bf54f6fba6be811e540b525e1d5cc716d14e73573386b2e3cce5ec2462369e9a656a48c2e31b7668352bfbc89a684de65a5c2a6e10ad4702dca6e7235e635bfe59d90561f79384b77140e608ce5647593f32d148b48742929335e88e8d720adde35a418cc1a743a2bdbc67b35e5401dcc04bfc4c0c8a9ddc579a9613f206f4aa802fa1fca58b863d3f2666d935f94b68e9d803b25570e42bb7fd48b82938b864bf2f00795be553cff04ea653fc3115546ce95c39bf04e6b100df09cab21c5ea19921bc8006869458fe837c2df0cf77f500e9cf7c55ffd0386505a4aecae3726c3c62b3ec'

## LZW

In [210]:
import lzw

# Example text to compress (can be any string)
original_text = ciphertext.hex()

# Convert the original text into bytes (UTF-8 encoding)
original_bytes = original_text.encode('utf-8')

# Part 1: Compression (in-memory)
compressed = lzw.compress(original_bytes)

# Convert compressed generator to a list to see its contents
compressed_list = list(compressed)

# Part 2: Decompression (in-memory)
decompressed_chunks = lzw.decompress(compressed_list)

# Join the decompressed chunks into a single byte string
decompressed_bytes = b"".join(decompressed_chunks)

# Convert decompressed bytes back to a string
decompressed_text = decompressed_bytes.decode('utf-8')

# Print each variable
print("Original Text (String):", original_text)
print("Original Bytes:", original_bytes)
print("Compressed List (Byte Chunks):", compressed_list)
print("Decompressed Bytes:", decompressed_bytes)
print("Decompressed Text (String):", decompressed_text)

# Part 3: Print sizes and results for comparison
original_size = len(original_bytes)
compressed_size = len(b"".join(compressed_list))  # Join to get compressed byte size
decompressed_size = len(decompressed_bytes)

# Print the sizes in bytes
print(f"\nOriginal size: {original_size} bytes")
print(f"Compressed size: {compressed_size} bytes")
print(f"Decompressed size: {decompressed_size} bytes")

# Check if decompression was successful
if decompressed_text == original_text:
    print("Decompression was successful!")
else:
    print("Decompression failed.")

Original Text (String): 04dd2a4b6cc7fc8addc7162361b4857a3010c24233c99f5f933e59dddc90f1586c9e1f14522af9bedae880b11948bf54f6fba6be811e540b525e1d5cc716d14e73573386b2e3cce5ec2462369e9a656a48c2e31b7668352bfbc89a684de65a5c2a6e10ad4702dca6e7235e635bfe59d90561f79384b77140e608ce5647593f32d148b48742929335e88e8d720adde35a418cc1a743a2bdbc67b35e5401dcc04bfc4c0c8a9ddc579a9613f206f4aa802fa1fca58b863d3f2666d935f94b68e9d803b25570e42bb7fd48b82938b864bf2f00795be553cff04ea653fc3115546ce95c39bf04e6b100df09cab21c5ea19921bc8006869458fe837c2df0cf77f500e9cf7c55ffd0386505a4aecae3726c3c62b3ec
Original Bytes: b'04dd2a4b6cc7fc8addc7162361b4857a3010c24233c99f5f933e59dddc90f1586c9e1f14522af9bedae880b11948bf54f6fba6be811e540b525e1d5cc716d14e73573386b2e3cce5ec2462369e9a656a48c2e31b7668352bfbc89a684de65a5c2a6e10ad4702dca6e7235e635bfe59d90561f79384b77140e608ce5647593f32d148b48742929335e88e8d720adde35a418cc1a743a2bdbc67b35e5401dcc04bfc4c0c8a9ddc579a9613f206f4aa802fa1fca58b863d3f2666d935f94b68e9d803b25570e42bb7fd48b82938b

## LSB

In [211]:
compressed_list

[b'\x18',
 b'\r',
 b'\x0c',
 b'\x86',
 b'A',
 b'\x91',
 b'\x84',
 b'h',
 b'b',
 b'\x1b',
 b'\x18',
 b'\xcc',
 b'c',
 b's',
 b'1',
 b'\x8c',
 b'p',
 b'a',
 b'\x82',
 b'C',
 b'\x06',
 b'#',
 b'a',
 b'\x90',
 b'\xcc',
 b'l',
 b'1',
 b'1',
 b'\r',
 b'\x07',
 b'\x03',
 b'Q',
 b'\xb9',
 b'\x84',
 b'f',
 b'0',
 b'\x18',
 b'\x8c',
 b'\x0c',
 b'c',
 b'!',
 b'\xa4',
 b'X',
 b'f',
 b'c',
 b'\x1c',
 b'\x8e',
 b'L',
 b'\xc3',
 b'S',
 b'0',
 b'\xe4',
 b'f',
 b'3',
 b'2',
 b'\x8d',
 b'G',
 b'0',
 b'C',
 b'$',
 b'\xa4',
 b'`',
 b'f',
 b'\x18',
 b'\x8d',
 b'G',
 b'\x10',
 b'\xa1',
 b'\xc9',
 b'\x94',
 b'c',
 b'9',
 b'\x1a',
 b'\r',
 b'F',
 b'P',
 b'i',
 b'q',
 b'\x88',
 b'\xca',
 b'd',
 b'0',
 b'\x99',
 b'G',
 b'\x03',
 b'\x81',
 b'\x81',
 b'\x88',
 b'b',
 b'1',
 b'\x1c',
 b'\xc6',
 b'\xcc',
 b'R',
 b'\xc1',
 b'\xa1',
 b'\x98',
 b'l',
 b'f',
 b'1',
 b'\x18',
 b'F',
 b'\xd4',
 b'q',
 b'\xc5',
 b'B',
 b'd',
 b'4',
 b'\xa7',
 b'P',
 b'\xc6',
 b'\xb3',
 b'\xf3',
 b' ',
 b'\xd6',
 b'\x16',
 b'7',
 b'\x8a',


In [212]:
embed_lsb("lsb_w_ECC_LZW.png", compressed_list)

In [213]:
extracted_data = extract_lsb("lsb_w_ECC_LZW.png")
extracted_data

'\x18\r\x0c\x86A\x91\x84hb\x1b\x18Ìcs1\x8cpa\x82C\x06#a\x90Ìl11\r\x07\x03Q¹\x84f0\x18\x8c\x0cc!¤Xfc\x1c\x8eLÃS0äf32\x8dG0C$¤`f\x18\x8dG\x10¡É\x94c9\x1a\rFPiq\x88Êd0\x99G\x03\x81\x81\x88b1\x1cÆÌRÁ¡\x98lf1\x18FÔqÅBd4§PÆ³ó Ö\x167\x8a\x19\x06#C(Üg\x1d\x98O\x0cC#,\xa0Ç22É\x06\x91X¼øs[\x1a\x8d\xa0ã\x89%Ú27\x1b\r\x87\x17\x01\x95PÅ\x0fÀbàfQ°ÔÃg\x83\r§ã\x08\x88Ðn0\x19M«vèµ\x90mpªL¦\x83\x91\x86\n\x807\x97\x8e!\x03{M\x87*0ÂÌ\x86Ùù\x98ÌÌ3Ñ[\x07\x11¡ÀÞL9\x19Kî\x14±Å,È7\x19g`\x97l¸Ðc\x851\x8cL<q\x99\x87\x1cdÈ\r\x86æ.`ÖÂ1\x9b\x18àUC\x18Ó×\x0f0Í&ÑÜ\x00æ1¿éV\x06\x86\x13\r42\x19\x86\x15\x00cf\x1cDð3\x19\x1f\x96(6\x19\x12ôµRB\\äÑM\x0c×@Õ\x1d\x0c\x06T\x98b\x18\x90Ñ\x91S\x0e\x1c\xa0Î\x06o\x15H\x000\x0c\x1b\x10ÕG\x85Ò\x81\x98f@\x86V\x05¿\x18Ã5B\x17_\x17\x80åg\x0cÃ\x95R1W\x12 Àd\x8c\x03\x98\x11t\x0cF5\x92\x02J\x83$e\x0f\x8a\x18°ÙRNÆe,3\rÒI\x11#\x19\x9bT²(\x19divJKFa\x900\x88Ùf¹\x07R\xa0EÙÑB\x92\x84Uå^\x90\x00'

In [214]:
byte_list = [bytes([ord(char)]) for char in extracted_data]
byte_list

[b'\x18',
 b'\r',
 b'\x0c',
 b'\x86',
 b'A',
 b'\x91',
 b'\x84',
 b'h',
 b'b',
 b'\x1b',
 b'\x18',
 b'\xcc',
 b'c',
 b's',
 b'1',
 b'\x8c',
 b'p',
 b'a',
 b'\x82',
 b'C',
 b'\x06',
 b'#',
 b'a',
 b'\x90',
 b'\xcc',
 b'l',
 b'1',
 b'1',
 b'\r',
 b'\x07',
 b'\x03',
 b'Q',
 b'\xb9',
 b'\x84',
 b'f',
 b'0',
 b'\x18',
 b'\x8c',
 b'\x0c',
 b'c',
 b'!',
 b'\xa4',
 b'X',
 b'f',
 b'c',
 b'\x1c',
 b'\x8e',
 b'L',
 b'\xc3',
 b'S',
 b'0',
 b'\xe4',
 b'f',
 b'3',
 b'2',
 b'\x8d',
 b'G',
 b'0',
 b'C',
 b'$',
 b'\xa4',
 b'`',
 b'f',
 b'\x18',
 b'\x8d',
 b'G',
 b'\x10',
 b'\xa1',
 b'\xc9',
 b'\x94',
 b'c',
 b'9',
 b'\x1a',
 b'\r',
 b'F',
 b'P',
 b'i',
 b'q',
 b'\x88',
 b'\xca',
 b'd',
 b'0',
 b'\x99',
 b'G',
 b'\x03',
 b'\x81',
 b'\x81',
 b'\x88',
 b'b',
 b'1',
 b'\x1c',
 b'\xc6',
 b'\xcc',
 b'R',
 b'\xc1',
 b'\xa1',
 b'\x98',
 b'l',
 b'f',
 b'1',
 b'\x18',
 b'F',
 b'\xd4',
 b'q',
 b'\xc5',
 b'B',
 b'd',
 b'4',
 b'\xa7',
 b'P',
 b'\xc6',
 b'\xb3',
 b'\xf3',
 b' ',
 b'\xd6',
 b'\x16',
 b'7',
 b'\x8a',


In [215]:
compressed_list == byte_list

True

In [216]:
# Part 2: Decompression (in-memory)
decompressed_chunks = lzw.decompress(byte_list)

# Join the decompressed chunks into a single byte string
decompressed_bytes = b"".join(decompressed_chunks)

# Convert decompressed bytes back to a string
decompressed_text = decompressed_bytes.decode('utf-8')

print("Decompressed Text (String):", decompressed_text)

Decompressed Text (String): 04dd2a4b6cc7fc8addc7162361b4857a3010c24233c99f5f933e59dddc90f1586c9e1f14522af9bedae880b11948bf54f6fba6be811e540b525e1d5cc716d14e73573386b2e3cce5ec2462369e9a656a48c2e31b7668352bfbc89a684de65a5c2a6e10ad4702dca6e7235e635bfe59d90561f79384b77140e608ce5647593f32d148b48742929335e88e8d720adde35a418cc1a743a2bdbc67b35e5401dcc04bfc4c0c8a9ddc579a9613f206f4aa802fa1fca58b863d3f2666d935f94b68e9d803b25570e42bb7fd48b82938b864bf2f00795be553cff04ea653fc3115546ce95c39bf04e6b100df09cab21c5ea19921bc8006869458fe837c2df0cf77f500e9cf7c55ffd0386505a4aecae3726c3c62b3ec


In [217]:
decompressed_text == ciphertext.hex()

True

In [218]:
cipher_data = bytes.fromhex(decompressed_text)
cipher_data

b'\x04\xdd*Kl\xc7\xfc\x8a\xdd\xc7\x16#a\xb4\x85z0\x10\xc2B3\xc9\x9f_\x93>Y\xdd\xdc\x90\xf1Xl\x9e\x1f\x14R*\xf9\xbe\xda\xe8\x80\xb1\x19H\xbfT\xf6\xfb\xa6\xbe\x81\x1eT\x0bR^\x1d\\\xc7\x16\xd1NsW3\x86\xb2\xe3\xcc\xe5\xec$b6\x9e\x9aejH\xc2\xe3\x1bvh5+\xfb\xc8\x9ahM\xe6Z\\*n\x10\xadG\x02\xdc\xa6\xe7#^c[\xfeY\xd9\x05a\xf7\x93\x84\xb7q@\xe6\x08\xceVGY?2\xd1H\xb4\x87B\x92\x935\xe8\x8e\x8dr\n\xdd\xe3ZA\x8c\xc1\xa7C\xa2\xbd\xbcg\xb3^T\x01\xdc\xc0K\xfcL\x0c\x8a\x9d\xdcW\x9a\x96\x13\xf2\x06\xf4\xaa\x80/\xa1\xfc\xa5\x8b\x86=?&f\xd95\xf9Kh\xe9\xd8\x03\xb2Up\xe4+\xb7\xfdH\xb8)8\xb8d\xbf/\x00y[\xe5S\xcf\xf0N\xa6S\xfc1\x15Tl\xe9\\9\xbf\x04\xe6\xb1\x00\xdf\t\xca\xb2\x1c^\xa1\x99!\xbc\x80\x06\x86\x94X\xfe\x83|-\xf0\xcfw\xf5\x00\xe9\xcf|U\xff\xd08e\x05\xa4\xae\xca\xe3rl<b\xb3\xec'

In [219]:
decrypted_message = decrypt(private_key_hex, cipher_data)
print("Decrypted Message:", decrypted_message.decode())

Decrypted Message: 3348610401970005#christofer*derian*budianto#tegal#1997-03-04#laki-laki#b#jl.*pala*22*no.*30#005#017#mejasem*tengah#kramat#katholik#belum*kawin#pelajar/mahasiswa#wni#seumur*hidup


In [220]:
decrypted_message.decode() == CONTENT

True

## Stego Metric

In [221]:
calculate_stego_metrics("lsb_w_ECC_LZW.png")

Metrics between original (ori-image/lena.png) and stego image (lsb_w_ECC_LZW.png):
MSE: 0.0021870930989583335
PSNR: 74.73213090631685
SSIM: 0.9999950870904414


(0.0021870930989583335, 74.73213090631685, 0.9999950870904414)

# LSB with Huffman

In [222]:
compressed_data

bytearray(b'\x02\xdfy>\xa8\x7f\x12G\xf7S\x11\x1e\xfb\xedYu\xcf\xc6)\x91\xd52\xe6\xc3\xa3usi\xf9\xd3OD;\xfb\xae\xa6\xf8\xde\xf8\x93\x80\xe1\xef\x03\x87\xed9\x90\xa2;\x10#\xf3\xe0\xdb\xca#\xbctG\xbe\x8f\xe9\xea\xa3%iQ\x9a[\xd1Wc&\xa6}\x835x\x0f\x8e\xd1\x01\xb5\x1c\x19^\xdd\xb2\x01d\xe5\x85J\x8a\xddr\x9e\x96\xcf\xcbF\xd5\xb9\x15>\xad\xd8')

In [223]:
compressed_data.hex()

'02df793ea87f1247f753111efbed5975cfc62991d532e6c3a3757369f9d34f443bfbaea6f8def89380e1ef0387ed3990a23b1023f3e0dbca23bc7447be8fe9eaa32569519a5bd1576326a67d8335780f8ed101b51c195eddb20164e5854a8add729e96cfcb46d5b9153eadd8'

In [224]:
embed_lsb("lsb_w_Huffman.png", compressed_data.hex())

In [225]:
extracted_data = extract_lsb("lsb_w_Huffman.png")
extracted_data

'02df793ea87f1247f753111efbed5975cfc62991d532e6c3a3757369f9d34f443bfbaea6f8def89380e1ef0387ed3990a23b1023f3e0dbca23bc7447be8fe9eaa32569519a5bd1576326a67d8335780f8ed101b51c195eddb20164e5854a8add729e96cfcb46d5b9153eadd8'

In [226]:
huffman_data = bytearray.fromhex(extracted_data)
huffman_data

bytearray(b'\x02\xdfy>\xa8\x7f\x12G\xf7S\x11\x1e\xfb\xedYu\xcf\xc6)\x91\xd52\xe6\xc3\xa3usi\xf9\xd3OD;\xfb\xae\xa6\xf8\xde\xf8\x93\x80\xe1\xef\x03\x87\xed9\x90\xa2;\x10#\xf3\xe0\xdb\xca#\xbctG\xbe\x8f\xe9\xea\xa3%iQ\x9a[\xd1Wc&\xa6}\x835x\x0f\x8e\xd1\x01\xb5\x1c\x19^\xdd\xb2\x01d\xe5\x85J\x8a\xddr\x9e\x96\xcf\xcbF\xd5\xb9\x15>\xad\xd8')

In [227]:
decompressed_data = huffman_decompress(huffman_data, huffman_tree)
print("Decompressed Data (as bytes):", decompressed_data)
print("Decompressed Data (as string):", decompressed_data.decode())

Decompressed Data (as bytes): bytearray(b'3348610401970005#christofer*derian*budianto#tegal#1997-03-04#laki-laki#b#jl.*pala*22*no.*30#005#017#mejasem*tengah#kramat#katholik#belum*kawin#pelajar/mahasiswa#wni#seumur*hidup')
Decompressed Data (as string): 3348610401970005#christofer*derian*budianto#tegal#1997-03-04#laki-laki#b#jl.*pala*22*no.*30#005#017#mejasem*tengah#kramat#katholik#belum*kawin#pelajar/mahasiswa#wni#seumur*hidup


In [228]:
decompressed_data.decode() == CONTENT

True

## Stego Metric

In [229]:
calculate_stego_metrics("lsb_w_Huffman.png")

Metrics between original (ori-image/lena.png) and stego image (lsb_w_Huffman.png):
MSE: 0.00107574462890625
PSNR: 77.81371174500211
SSIM: 0.9999984186394683


(0.00107574462890625, 77.81371174500211, 0.9999984186394683)

# LSB with ECC, Huffman

## ECC

In [230]:
ciphertext.hex()

'04dd2a4b6cc7fc8addc7162361b4857a3010c24233c99f5f933e59dddc90f1586c9e1f14522af9bedae880b11948bf54f6fba6be811e540b525e1d5cc716d14e73573386b2e3cce5ec2462369e9a656a48c2e31b7668352bfbc89a684de65a5c2a6e10ad4702dca6e7235e635bfe59d90561f79384b77140e608ce5647593f32d148b48742929335e88e8d720adde35a418cc1a743a2bdbc67b35e5401dcc04bfc4c0c8a9ddc579a9613f206f4aa802fa1fca58b863d3f2666d935f94b68e9d803b25570e42bb7fd48b82938b864bf2f00795be553cff04ea653fc3115546ce95c39bf04e6b100df09cab21c5ea19921bc8006869458fe837c2df0cf77f500e9cf7c55ffd0386505a4aecae3726c3c62b3ec'

## Huffman

In [231]:
# Example Usage of Huffman Compression/Decompression

# Step 1: The original plaintext data (as a byte string)
plaintext = ciphertext.hex().encode()

# Print the original data
print("Original Data (as bytes):", plaintext)
print("Original Data (as string):", plaintext.decode())

# Step 2: Compress the plaintext data using Huffman encoding
compressed_data, huffman_tree = huffman_compress(plaintext)
print("Compressed Data (in bytes):", compressed_data)

Original Data (as bytes): b'04dd2a4b6cc7fc8addc7162361b4857a3010c24233c99f5f933e59dddc90f1586c9e1f14522af9bedae880b11948bf54f6fba6be811e540b525e1d5cc716d14e73573386b2e3cce5ec2462369e9a656a48c2e31b7668352bfbc89a684de65a5c2a6e10ad4702dca6e7235e635bfe59d90561f79384b77140e608ce5647593f32d148b48742929335e88e8d720adde35a418cc1a743a2bdbc67b35e5401dcc04bfc4c0c8a9ddc579a9613f206f4aa802fa1fca58b863d3f2666d935f94b68e9d803b25570e42bb7fd48b82938b864bf2f00795be553cff04ea653fc3115546ce95c39bf04e6b100df09cab21c5ea19921bc8006869458fe837c2df0cf77f500e9cf7c55ffd0386505a4aecae3726c3c62b3ec'
Original Data (as string): 04dd2a4b6cc7fc8addc7162361b4857a3010c24233c99f5f933e59dddc90f1586c9e1f14522af9bedae880b11948bf54f6fba6be811e540b525e1d5cc716d14e73573386b2e3cce5ec2462369e9a656a48c2e31b7668352bfbc89a684de65a5c2a6e10ad4702dca6e7235e635bfe59d90561f79384b77140e608ce5647593f32d148b48742929335e88e8d720adde35a418cc1a743a2bdbc67b35e5401dcc04bfc4c0c8a9ddc579a9613f206f4aa802fa1fca58b863d3f2666d935f94b68e9d803b25570e42

## LSB

In [232]:
compressed_data

bytearray(b'\x08x"S\x86\xde\xe0\xae\x93"\xe0\x1d\\\xd1h\x9f\x03\xc7\x17\xe5\x85\xcc\xe4J\xfaL\xcb\xf4".G\xa1\xf9\xdeK\x1a\x18\xf5S\xa4k#\xb9\x97a\x14\x89j\xf8\xad\xa6=k\x91\x1b\xf8v\xf5\xfb\x12\xfe\xe0\x1d!\x8b\x0c\xf0\xcc\x9de\xbc\xee\xbf\xbeX\xd5\xcdKC\xdf\xd3\x89\xe5\xbc\x16\r\xd9\xcfV\xa6\xe9C\xd9\x82\xbd\xf3\xfeS\xdb\x172\x80u.=\xb0\\\xfb\xdc\xf6\xab\xf4$\x7f\xd1\xa0L\x98`\x01\x87\xbdy\xeb\xfd\x80\xf4\xca\xc5!\x89h\x90\x85EL\xcf\xb9\x9b\x92\x05s"\xbc\xf3\x81\x9e\xe10\x8c5bn\xd0l\xfb\xf8q.\xe7\x86\xae\x8e~\x93B.\xf0CM\x1c\xa5}\xa83\x97Z1\xae?\x96\x9d\xc2\xca]\xdd$\xcf\xa4\x86\xd9\xb4)|e\xff\x07\xb8V`\xa2\x89iT\xc9i\xd8jZw\x04\xf6\xbf\xfc\xea\xa7\x8b=\xfc\xae\xc1\x1f\xf8\xde\xb4\xfe\xc4jx\xbdaw*t\xe3e\x1e\xfb1DQn\x97}\x9dH\xf9\xab\x9c\x0eR\xa7\xea\x00\xafw\xb4\xea\x0e\xff\xaa\'\xc9\xdf\x7f8;\xe3\xbc\x05\xde\xce\xd5l\xbe\x00')

In [233]:
compressed_data.hex()

'0878225386dee0ae9322e01d5cd1689f03c717e585cce44afa4ccbf4222e47a1f9de4b1a18f553a46b23b9976114896af8ada63d6b911bf876f5fb12fee01d218b0cf0cc9d65bceebfbe58d5cd4b43dfd389e5bc160dd9cf56a6e943d982bdf3fe53db173280752e3db05cfbdcf6abf4247fd1a04c98600187bd79ebfd80f4cac52189689085454ccfb99b92057322bcf3819ee1308c35626ed06cfbf8712ee786ae8e7e93422ef0434d1ca57da833975a31ae3f969dc2ca5ddd24cfa486d9b4297c65ff07b85660a2896954c969d86a5a7704f6bffceaa78b3dfcaec11ff8deb4fec46a78bd61772a74e3651efb3144516e977d9d48f9ab9c0e52a7ea00af77b4ea0effaa27c9df7f383be3bc05deced56cbe00'

In [234]:
embed_lsb('LSB_w_ECC_Huffman.png', compressed_data.hex())

In [236]:
extracted_data = extract_lsb('LSB_w_ECC_Huffman.png')
extracted_data

'0878225386dee0ae9322e01d5cd1689f03c717e585cce44afa4ccbf4222e47a1f9de4b1a18f553a46b23b9976114896af8ada63d6b911bf876f5fb12fee01d218b0cf0cc9d65bceebfbe58d5cd4b43dfd389e5bc160dd9cf56a6e943d982bdf3fe53db173280752e3db05cfbdcf6abf4247fd1a04c98600187bd79ebfd80f4cac52189689085454ccfb99b92057322bcf3819ee1308c35626ed06cfbf8712ee786ae8e7e93422ef0434d1ca57da833975a31ae3f969dc2ca5ddd24cfa486d9b4297c65ff07b85660a2896954c969d86a5a7704f6bffceaa78b3dfcaec11ff8deb4fec46a78bd61772a74e3651efb3144516e977d9d48f9ab9c0e52a7ea00af77b4ea0effaa27c9df7f383be3bc05deced56cbe00'

In [237]:
huffman_data = bytearray.fromhex(extracted_data)
huffman_data

bytearray(b'\x08x"S\x86\xde\xe0\xae\x93"\xe0\x1d\\\xd1h\x9f\x03\xc7\x17\xe5\x85\xcc\xe4J\xfaL\xcb\xf4".G\xa1\xf9\xdeK\x1a\x18\xf5S\xa4k#\xb9\x97a\x14\x89j\xf8\xad\xa6=k\x91\x1b\xf8v\xf5\xfb\x12\xfe\xe0\x1d!\x8b\x0c\xf0\xcc\x9de\xbc\xee\xbf\xbeX\xd5\xcdKC\xdf\xd3\x89\xe5\xbc\x16\r\xd9\xcfV\xa6\xe9C\xd9\x82\xbd\xf3\xfeS\xdb\x172\x80u.=\xb0\\\xfb\xdc\xf6\xab\xf4$\x7f\xd1\xa0L\x98`\x01\x87\xbdy\xeb\xfd\x80\xf4\xca\xc5!\x89h\x90\x85EL\xcf\xb9\x9b\x92\x05s"\xbc\xf3\x81\x9e\xe10\x8c5bn\xd0l\xfb\xf8q.\xe7\x86\xae\x8e~\x93B.\xf0CM\x1c\xa5}\xa83\x97Z1\xae?\x96\x9d\xc2\xca]\xdd$\xcf\xa4\x86\xd9\xb4)|e\xff\x07\xb8V`\xa2\x89iT\xc9i\xd8jZw\x04\xf6\xbf\xfc\xea\xa7\x8b=\xfc\xae\xc1\x1f\xf8\xde\xb4\xfe\xc4jx\xbdaw*t\xe3e\x1e\xfb1DQn\x97}\x9dH\xf9\xab\x9c\x0eR\xa7\xea\x00\xafw\xb4\xea\x0e\xff\xaa\'\xc9\xdf\x7f8;\xe3\xbc\x05\xde\xce\xd5l\xbe\x00')

In [238]:
decompressed_data = huffman_decompress(huffman_data, huffman_tree)
print("Decompressed Data (as bytes):", decompressed_data)
print("Decompressed Data (as string):", decompressed_data.decode())

Decompressed Data (as bytes): bytearray(b'04dd2a4b6cc7fc8addc7162361b4857a3010c24233c99f5f933e59dddc90f1586c9e1f14522af9bedae880b11948bf54f6fba6be811e540b525e1d5cc716d14e73573386b2e3cce5ec2462369e9a656a48c2e31b7668352bfbc89a684de65a5c2a6e10ad4702dca6e7235e635bfe59d90561f79384b77140e608ce5647593f32d148b48742929335e88e8d720adde35a418cc1a743a2bdbc67b35e5401dcc04bfc4c0c8a9ddc579a9613f206f4aa802fa1fca58b863d3f2666d935f94b68e9d803b25570e42bb7fd48b82938b864bf2f00795be553cff04ea653fc3115546ce95c39bf04e6b100df09cab21c5ea19921bc8006869458fe837c2df0cf77f500e9cf7c55ffd0386505a4aecae3726c3c62b3ec')
Decompressed Data (as string): 04dd2a4b6cc7fc8addc7162361b4857a3010c24233c99f5f933e59dddc90f1586c9e1f14522af9bedae880b11948bf54f6fba6be811e540b525e1d5cc716d14e73573386b2e3cce5ec2462369e9a656a48c2e31b7668352bfbc89a684de65a5c2a6e10ad4702dca6e7235e635bfe59d90561f79384b77140e608ce5647593f32d148b48742929335e88e8d720adde35a418cc1a743a2bdbc67b35e5401dcc04bfc4c0c8a9ddc579a9613f206f4aa802fa1fca58b863d3f2666d935f9

In [239]:
cipher_data = bytes.fromhex(decompressed_data.decode())
cipher_data

b'\x04\xdd*Kl\xc7\xfc\x8a\xdd\xc7\x16#a\xb4\x85z0\x10\xc2B3\xc9\x9f_\x93>Y\xdd\xdc\x90\xf1Xl\x9e\x1f\x14R*\xf9\xbe\xda\xe8\x80\xb1\x19H\xbfT\xf6\xfb\xa6\xbe\x81\x1eT\x0bR^\x1d\\\xc7\x16\xd1NsW3\x86\xb2\xe3\xcc\xe5\xec$b6\x9e\x9aejH\xc2\xe3\x1bvh5+\xfb\xc8\x9ahM\xe6Z\\*n\x10\xadG\x02\xdc\xa6\xe7#^c[\xfeY\xd9\x05a\xf7\x93\x84\xb7q@\xe6\x08\xceVGY?2\xd1H\xb4\x87B\x92\x935\xe8\x8e\x8dr\n\xdd\xe3ZA\x8c\xc1\xa7C\xa2\xbd\xbcg\xb3^T\x01\xdc\xc0K\xfcL\x0c\x8a\x9d\xdcW\x9a\x96\x13\xf2\x06\xf4\xaa\x80/\xa1\xfc\xa5\x8b\x86=?&f\xd95\xf9Kh\xe9\xd8\x03\xb2Up\xe4+\xb7\xfdH\xb8)8\xb8d\xbf/\x00y[\xe5S\xcf\xf0N\xa6S\xfc1\x15Tl\xe9\\9\xbf\x04\xe6\xb1\x00\xdf\t\xca\xb2\x1c^\xa1\x99!\xbc\x80\x06\x86\x94X\xfe\x83|-\xf0\xcfw\xf5\x00\xe9\xcf|U\xff\xd08e\x05\xa4\xae\xca\xe3rl<b\xb3\xec'

In [240]:
decrypted_message = decrypt(private_key_hex, cipher_data)
print("Decrypted Message:", decrypted_message.decode())

Decrypted Message: 3348610401970005#christofer*derian*budianto#tegal#1997-03-04#laki-laki#b#jl.*pala*22*no.*30#005#017#mejasem*tengah#kramat#katholik#belum*kawin#pelajar/mahasiswa#wni#seumur*hidup


In [241]:
decrypted_message.decode() == CONTENT

True

## Stego Metric

In [242]:
calculate_stego_metrics('LSB_w_ECC_Huffman.png')

Metrics between original (ori-image/lena.png) and stego image (LSB_w_ECC_Huffman.png):
MSE: 0.002719879150390625
PSNR: 73.78530752967707
SSIM: 0.9999931274953937


(0.002719879150390625, 73.78530752967707, 0.9999931274953937)